In [61]:
import pandas as pd
import numpy as np

data = pd.read_csv("../data/goodreads_library_export.csv")
data.head()

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
0,32492682,Mroczna materia,Blake Crouch,"Crouch, Blake",Paweł Wieczorek,"=""8365521784""","=""9788365521781""",0,4.14,Zysk i S-ka,...,NaN,2025/04/30,to-read,to-read (#34),to-read,NaN,NaN,NaN,0,0
1,53218051,Odzyskanie czasu,Baoshu,"Baoshu, Baoshu",Jan Andrzejewski,"=""8380625293""","=""9788380625297""",0,3.64,Dom Wydawniczy Rebis,...,NaN,2025/04/22,to-read,to-read (#33),to-read,NaN,NaN,NaN,0,0
2,188606529,Neuromania - Le vrai du faux sur votre cerveau,Albert Moukheiber,"Moukheiber, Albert",NaN,"=""2370734310""","=""9782370734310""",0,4.28,ALLARY,...,NaN,2025/04/22,to-read,to-read (#32),to-read,NaN,NaN,NaN,0,0
3,228868428,Kaori,Marta Sobiecka,"Sobiecka, Marta",NaN,"=""8397352198""","=""9788397352193""",0,3.52,Pulp Books,...,NaN,2025/04/22,to-read,to-read (#31),to-read,NaN,NaN,NaN,0,0
4,223876013,Intermezzo,Sally Rooney,"Rooney, Sally",Jerzy Kozłowski,"=""8383873042""","=""9788383873046""",0,3.88,Wydawnictwo W.A.B.,...,NaN,2025/04/16,to-read,to-read (#30),to-read,NaN,NaN,NaN,0,0


Deleting unnecessary columns + unified columns name (like in dataset), also I need to merge all authors (i think) and change ISBN to int

In [62]:
columns = ["Book Id", "Title", "Author", "Additional Authors", "ISBN", "My Rating", "Read Count"]
data = data.loc[:, columns]

lower_cols = [col.lower().replace(' ', '_') for col in columns]
mapping = dict(zip(columns,lower_cols))
data.rename(columns=mapping, inplace=True)
data['additional_authors'] = data['additional_authors'].fillna('')
data['author'] = data['author'] + ', ' + data['additional_authors']
data.drop(columns=['additional_authors'], inplace=True)


data['isbn'] = data['isbn'].str.replace(r'["=]', '', regex=True)
display(data)

,book_id,title,author,isbn,my_rating,read_count
0,32492682,Mroczna materia,"Blake Crouch, Paweł Wieczorek",8365521784,0,0
1,53218051,Odzyskanie czasu,"Baoshu, Jan Andrzejewski",8380625293,0,0
2,188606529,Neuromania - Le vrai du faux sur votre cerveau,"Albert Moukheiber,",2370734310,0,0
3,228868428,Kaori,"Marta Sobiecka,",8397352198,0,0
4,223876013,Intermezzo,"Sally Rooney, Jerzy Kozłowski",8383873042,0,0
...,...,...,...,...,...,...
89,2,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",,5,1
90,6,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",0439139597,0,1
91,5,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré",043965548X,0,1
92,15881,Harry Potter and the Chamber of Secrets (Harry...,"J.K. Rowling, Mary GrandPré",,0,1


Now we're creating table with tags just like in dataset before

In [63]:
tags = pd.read_csv('../data/tags.csv')
help_table = pd.read_csv('../data/book_tags.csv')
avg_popularity = np.mean(help_table['count'])

help_table = (
    help_table[help_table['count'] > avg_popularity]
    .rename(columns={'goodreads_book_id': 'book_id'})
    .drop(columns='count')
)
help_table = help_table.merge(tags, on='tag_id', how='left')
merged_tags = help_table.groupby('book_id')['tag_name'].apply(list).reset_index()

merged_tags.head()

,book_id,tag_name
0,1,"[to-read, fantasy, favorites, currently-readin..."
1,2,"[to-read, currently-reading, fantasy, favorite..."
2,3,"[to-read, favorites, fantasy, currently-readin..."
3,5,"[favorites, fantasy, currently-reading, young-..."
4,6,"[fantasy, young-adult, fiction, harry-potter, ..."


In [64]:
user_dataset = data.merge(merged_tags, on='book_id', how='left')
user_dataset['tag_name'] = user_dataset['tag_name'].fillna('')

display(user_dataset)

,book_id,title,author,isbn,my_rating,read_count,tag_name
0,32492682,Mroczna materia,"Blake Crouch, Paweł Wieczorek",8365521784,0,0,
1,53218051,Odzyskanie czasu,"Baoshu, Jan Andrzejewski",8380625293,0,0,
2,188606529,Neuromania - Le vrai du faux sur votre cerveau,"Albert Moukheiber,",2370734310,0,0,
3,228868428,Kaori,"Marta Sobiecka,",8397352198,0,0,
4,223876013,Intermezzo,"Sally Rooney, Jerzy Kozłowski",8383873042,0,0,
...,...,...,...,...,...,...,...
89,2,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",,5,1,"[to-read, currently-reading, fantasy, favorite..."
90,6,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",0439139597,0,1,"[fantasy, young-adult, fiction, harry-potter, ..."
91,5,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré",043965548X,0,1,"[favorites, fantasy, currently-reading, young-..."
92,15881,Harry Potter and the Chamber of Secrets (Harry...,"J.K. Rowling, Mary GrandPré",,0,1,"[fantasy, favorites, currently-reading, young-..."
